## Hello World, AutoGen

Import namespace

In [7]:
from dataclasses import dataclass
from typing import List

from autogen_core.base import MessageContext
from autogen_core.components import DefaultTopicId, RoutedAgent, default_subscription, message_handler
from autogen_core.components.code_executor import CodeExecutor, extract_markdown_code_blocks
from autogen_core.components.models import (
    AssistantMessage,
    ChatCompletionClient,
    LLMMessage,
    SystemMessage,
    UserMessage,
)
import tempfile

from autogen_core.application import SingleThreadedAgentRuntime
from autogen_core.components.models import OpenAIChatCompletionClient

## Create a Message data class
The `Message` class is a simple data class that holds a text content.

In [2]:
@dataclass
class Message:
    content: str

## Create your first AutoGen Assistant Agent

The assistant will be invoked when it receives a `Message` from agent runtime. Then it will use LLM to generate a response message and publish it back to the agent runtime.

The assistant has `default_subscription`, which means it subscribles to messages with all topics from the agent runtime.

In [10]:
@default_subscription
class Assistant(RoutedAgent):
    def __init__(self, model_client: ChatCompletionClient) -> None:
        super().__init__("An assistant agent.")
        self._model_client = model_client
        self._chat_history: List[LLMMessage] = [
            SystemMessage(
                content="""
            You are a helpful AI assistant. You greet the user by saying 'Hello World <name>! How can I help you today?'.
            """,
            )
        ]

    @message_handler
    async def handle_message(self, message: Message, ctx: MessageContext) -> None:
        self._chat_history.append(UserMessage(content=message.content, source="user"))
        result = await self._model_client.create(self._chat_history)
        print(f"\n{'-'*80}\nAssistant:\n{result.content}")
        self._chat_history.append(AssistantMessage(content=result.content, source="assistant"))  # type: ignore
        await self.publish_message(Message(content=result.content), DefaultTopicId())  # type: ignore

Create  a single thread agent runtime

In [11]:
# Create an local embedded runtime.
import os;
runtime = SingleThreadedAgentRuntime()
await Assistant.register(
        runtime,
        "assistant",
        lambda: Assistant(
            OpenAIChatCompletionClient(
                model="gpt-4o",
                # get api key from env:OPENAI_API_KEY
                api_key=os.environ.get("OPENAI_API_KEY"),
            )
        ),
    )

AgentType(type='assistant')

## Send a message to agent

In [12]:
# Start the runtime and publish a message to the assistant.
runtime.start()
await runtime.publish_message(
    Message("Hello, I am Geeno"), DefaultTopicId()
)
await runtime.stop_when_idle()


--------------------------------------------------------------------------------
Assistant:
Hello World Geeno! How can I help you today?


C:\Users\xiaoyuz\AppData\Local\Temp\ipykernel_84428\3394563498.py:17: UserWarning: Resolved model mismatch: gpt-4o-2024-05-13 != gpt-4o-2024-08-06. Model mapping may be incorrect.
  result = await self._model_client.create(self._chat_history)


## Conclusion

Congratulations! You have successfully created your first AutoGen Assistant Agent and sent a message to it.

## What we have learned
- How to create a simple data class
- How to create an AutoGen Assistant Agent
- How to create and start an agent runtime
- How to register an agent to the agent runtime
- How to send a message to an agent runtime and receive a response